In [ ]:
import pandas as pd
import numpy as np

#setting up a file dir so I can reuse it
file_dir = '/Users/richbafa/Desktop/novelis_take_home/take_home_docs/'

#reading in the data for EDA
user_trans =  pd.read_csv(file_dir + 'Candidate_tech_evaluation_candidate_copy_data science_fraud.csv')
ip_to_country_map = pd.read_excel(file_dir + 'Candidate_tech_evaluation_candidate_copy_datascience_IpAddress_to_Country.xlsx')



# Get a list of unique countries to loop over it
countries = list(ip_to_country_map.country.unique())

# DataFrame for unique IP address
ip_table = user_trans[['ip_address']].drop_duplicates()
ip_cross_join = ip_table.merge(ip_to_country_map, how = 'cross')
ip_cross_join = ip_cross_join[(ip_cross_join.ip_address >= ip_cross_join.lower_bound_ip_address) & (ip_cross_join.ip_address <= ip_cross_join.upper_bound_ip_address)]


In [ ]:
#for compute efficiency we have to initialize a dict and append each cross join to the dict before converting back to dataframe
results = []
i = 1
total_iter = len(countries)
for country in countries:
    
    print('iteration {0} of {1}'.format(i, total_iter))
    
    #subset the country map df to include only one country and do the cross join
    country_df = ip_to_country_map[ip_to_country_map['country'] == country]
    
    tmp_cross_join = ip_table.merge(country_df, how = 'cross')
    tmp_cross_join =  tmp_cross_join[(tmp_cross_join.ip_address >= tmp_cross_join.lower_bound_ip_address) & (tmp_cross_join.ip_address <= tmp_cross_join.upper_bound_ip_address)]
    
    # Store the cross join as a list of dict and append it to the master list initialized above
    # Before converting to list of dict, I want to make sure I am dropping duplicates
    tmp = tmp_cross_join[['ip_address', 'country']].drop_duplicates().to_dict('records')
    results.append(tmp)
    
    i += 1

# convert the list of dict bact to Dataframe
ip_look_up = pd.DataFrame.from_dict(results)
ip_look_up.to_csv(file_dir + 'ip_look_up.csv')